In [ ]:
!pip install pip

import pip

for each in ["requests","beautifulsoup4"]: 
    pip.main(['install', each])

import requests
from bs4 import BeautifulSoup


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#scraping indeed for positions according to candidate profile
import datetime, time
!pip install wget
import wget

import csv, re
import logging

import pandas as pd
import sqlite3
import json
from pytz import timezone

from requests.auth import HTTPBasicAuth
import requests
from requests.exceptions import HTTPError

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
client_key = "77zk7frrida7k3"
client_secret = "CCISHQyTUjP3xqvZ"

In [ ]:
job_positions = ["data%20scientist%20intern","software%20engineering%20intern","Devops%20intern","Software%20Engineer","Data%20Analyst","Data%20Engineer"]
headers = {'Accept': 'application/json'}
auth = HTTPBasicAuth(client_key, client_secret)
job_url_template = "https://www.linkedin.com/jobs/view/"
job_df = pd.DataFrame(columns=["job_title","industry","company","description","job_locality","job_region","skills","job_post_expiry","education_requirements", "base_salary_min","base_salary_max","base_salary_currency","url"])

In [ ]:
list_of_links = []
try:
    s = requests.Session()
  
    for position in job_positions:
        for i in range(0,500,25):
        #for i in range(0,1):
            linkedin_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords='+position+\
                            '&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&position=8&pageNum=0&currentJobId=3257884433&start='+str(i)

            time.sleep(1)
            html_text = s.get(linkedin_url, headers=headers).text
            soup = BeautifulSoup(html_text, "html.parser")
            count=0
            for meat in soup.find_all('a'):
                if (meat.get('href') != None):
                    if(job_url_template in meat.get('href')):
                        list_of_links.append(meat.get('href'))
                        count+=1
    print(len(list_of_links))
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

2999


In [ ]:
job_data = []
print(len(list_of_links))
for i in range(0,len(list_of_links)):
    job_req = requests.Session()
    job_url=list_of_links[i]

    job_text = job_req.get(job_url, headers=headers, auth=auth).text
    job_soup = BeautifulSoup(job_text, "html.parser")
    job_desc_soup = BeautifulSoup(job_text, "lxml")
    job_details_content = job_soup.find('script',  {"type": "application/ld+json"})
    job_desc = job_desc_soup.find('div', class_='description__text')

    if(job_details_content and job_details_content.text):
        job_json = json.loads(job_details_content.text)
        job_details = [job_json.get('title'), job_json.get('industry')]

        if(job_json.get('identifier')):
            job_details.append(job_json.get('identifier').get('name'))
        else:
            job_details.append(None)
        
        if(job_desc):
            job_details.append(job_desc.text.replace("Show more"," ").replace("Show less", " "))
        else:
            job_details.append(None)
        
        if('jobLocation' in job_json and 'address' in job_json.get('jobLocation')):
            job_details.extend( [job_json.get('jobLocation').get('address').get('addressLocality'), 
                                job_json.get('jobLocation').get('address').get('addressRegion')])
        else:
            job_details.extend([None,None])
            
        job_details.extend( [job_json.get('skills'), job_json.get('validThrough')])

        if('educationRequirements' in job_json):
            job_details.append(job_json.get('educationRequirements').get('credentialCategory'))
        else:
            job_details.append(None)

        if('baseSalary' in job_json and 'value' in job_json.get('baseSalary')):
            job_details.extend([job_json.get('baseSalary').get('value').get('minValue'),
                      job_json.get('baseSalary').get('value').get('maxValue'), job_json.get('baseSalary').get('currency')])
        else:
            job_details.extend([None,None,None])
        job_details.append(job_url)
        
        job_data.append(job_details)
        time.sleep(1.25)
    else:
        print(job_url)
        


2999
https://www.linkedin.com/jobs/view/data-science-virtual-internship-30-dec-march-2023-at-data-glacier-3388699447?refId=3qF6PkYfVQnIPwqQPuzdzg%3D%3D&trackingId=TQ63ssZdgwKh691UewSJww%3D%3D&position=14&pageNum=0&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/data-scientist-opportunity-for-working-remotely-at-vmware-3371085138?refId=aF9Zqn5V0%2BFF49W%2FqWYFdQ%3D%3D&trackingId=UpYGb0ozTREXhYB%2FReQFKw%3D%3D&position=8&pageNum=1&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/data-scientist-at-gvd-systems-llc-3386036250?refId=jnOxBBqnnRRG4DpJgeUoGg%3D%3D&trackingId=7MEpNeFgwO6DHVeH2BGaWw%3D%3D&position=3&pageNum=5&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/data-scientist-at-covetus-3388709220?refId=jnOxBBqnnRRG4DpJgeUoGg%3D%3D&trackingId=MFqXWDPGErPSNvKjoTjdPA%3D%3D&position=10&pageNum=5&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/data-scientist-at-awra-us-3388639179

In [ ]:
job_df = pd.DataFrame(job_data, 
                       columns=["job_title","industry","company","description", "job_locality","job_region","skills","job_post_expiry","education_requirements", "base_salary_min","base_salary_max","base_salary_currency","url"])

job_df

,job_title,industry,company,description,job_locality,job_region,skills,job_post_expiry,education_requirements,base_salary_min,base_salary_max,base_salary_currency,url
0,Data Scientist Internship,Radio and Television Broadcasting,iHeartMedia,\n\n\n iHeartMediaCurrent employees and...,Texas,United States,,2022-12-20T17:38:56.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
1,Data Scientist Intern,Financial Services,Mercedes-Benz Financial Services USA LLC,\n\n\nTasks Internship Job Title: Data Scienti...,Farmington Hills,MI,,2022-12-28T19:46:33.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
2,Data Scientist Internship - OR (US),Insurance,Asurion,\n\n\n 2022-75202Asurion Data Science I...,Nashville,TN,,2023-01-04T03:56:32.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
3,Data Scientist Internship - ML (US),Insurance,Asurion,\n\n\n 2022-75205Asurion Data Science I...,Nashville,TN,,2023-01-04T03:56:32.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
4,Data Scientist Intern (Remote),"Appliances, Electrical, and Electronics Manufa...",Digi-Key Electronics,\n\n\n Digi-Key is one of the fastest g...,Home,MN,,2022-12-18T19:25:54.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2804,Data Engineer,Construction Hardware Manufacturing,"Lowe's Companies, Inc.",\n\n\nJob SummaryThe primary purpose of this r...,Georgia,United States,,2022-12-29T03:07:14.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2805,Data Engineer,"IT Services and IT Consulting,Motor Vehicle Ma...",HTC Global Services,\n\n\nHTC Global Services wants you. Come buil...,Dearborn,MI,,2022-12-16T21:01:59.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2806,Data Engineer II,Higher Education,Tulane University,\n\n\n Data Engineer IIVP Technology/In...,New Orleans,LA,,2022-12-30T09:15:09.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2807,Data Engineer,IT Services and IT Consulting,NucleusTeq,\n\n\nRole InfoHands on BI Data Engineer role ...,Virginia,United States,,2022-12-28T09:01:19.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...


In [ ]:
job_df['description']

0    iHeartMediaCurrent employees and contingent wo...
1    Tasks Internship Job Title: Data Scientist Int...
2    2022-75202Asurion Data Science Internship - OR...
3    2022-75205Asurion Data Science Internship - ML...
4    Digi-Key is one of the fastest growing distrib...
Name: description, dtype: object

In [ ]:
print(job_data)

In [ ]:
job_df = pd.DataFrame(job_data, 
                       columns=["job_title","industry","company","description", "job_locality","job_region","skills","job_post_expiry","education_requirements", "base_salary_min","base_salary_max","base_salary_currency","url"])
outfile="jobs_data"
outfile=outfile+'.csv'
print(outfile)
job_df.to_csv(outfile, sep=',', encoding='utf-8')

job_df

jobs_data.csv


,job_title,industry,company,description,job_locality,job_region,skills,job_post_expiry,education_requirements,base_salary_min,base_salary_max,base_salary_currency,url
0,Data Scientist Internship,Radio and Television Broadcasting,iHeartMedia,\n\n\n iHeartMediaCurrent employees and...,Texas,United States,,2022-12-20T17:38:56.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
1,Data Scientist Intern,Financial Services,Mercedes-Benz Financial Services USA LLC,\n\n\nTasks Internship Job Title: Data Scienti...,Farmington Hills,MI,,2022-12-28T19:46:33.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
2,Data Scientist Internship - OR (US),Insurance,Asurion,\n\n\n 2022-75202Asurion Data Science I...,Nashville,TN,,2023-01-04T03:56:32.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
3,Data Scientist Internship - ML (US),Insurance,Asurion,\n\n\n 2022-75205Asurion Data Science I...,Nashville,TN,,2023-01-04T03:56:32.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
4,Data Scientist Intern (Remote),"Appliances, Electrical, and Electronics Manufa...",Digi-Key Electronics,\n\n\n Digi-Key is one of the fastest g...,Home,MN,,2022-12-18T19:25:54.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2804,Data Engineer,Construction Hardware Manufacturing,"Lowe's Companies, Inc.",\n\n\nJob SummaryThe primary purpose of this r...,Georgia,United States,,2022-12-29T03:07:14.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2805,Data Engineer,"IT Services and IT Consulting,Motor Vehicle Ma...",HTC Global Services,\n\n\nHTC Global Services wants you. Come buil...,Dearborn,MI,,2022-12-16T21:01:59.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2806,Data Engineer II,Higher Education,Tulane University,\n\n\n Data Engineer IIVP Technology/In...,New Orleans,LA,,2022-12-30T09:15:09.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2807,Data Engineer,IT Services and IT Consulting,NucleusTeq,\n\n\nRole InfoHands on BI Data Engineer role ...,Virginia,United States,,2022-12-28T09:01:19.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
